# Scribbles Notebook 
> A bunch of very different topics scribbles 

- toc: true 
- badges: true
- comments: true
- categories: [jupyter,deeplearning,python]


## Transformers 

To deal with sequential data we have to options:
   - 1-D convolution NN
       - processing can be parallel 
       - not practical for long sequences 
   - Recurrent NN 
       - can't happen in prallel 
       - have gradient vanshing problem of the squence becomes so long
       - we have bottleneck at the end of the encoder 
   - RNN with attention mechanism 
       - to solve the bottleneck problem, we make Encoder-Decoder attention 
       - Decoder utilzes:
           - context vector 
           - weighted sum of hidden states (h1,h2, ... ) from the encoder 

### Transformers 

#### Encoder
- first we do input embedding, and positional embedding 
- in self attention: we multiply q,w,v by a matrix to do lenear transformation
- self attentoion: k * q --> scaling down --> softmax --> * v 
 
### multi-head  attention 
- works as we use many filters in CNN  
- in wide attention: it takes every word and spread it multi-head attention
- in narrow attention:  it take every word and split it up across the multi-head 
    - but didnt we lose the adcantage of using multi-head as mutli prespectives, as we do with filters in CNN?

### Positional info

- positional encoding using the rotation sin/cos matrix 
- positional embedding 

### Residual connections
- to give the chance to skip some learning parameters if it's better to minimize the loss 

### Layer Normalization 

- in batch normalization 
    - ==> we normalize to zero mean and unity varince 
    - we calculate for all samples in each batch (for each channel )
- in layer normalization 
    - ==>  $y = \gamma * x +  \beta $  where gamm and bata are trainable parametes
    - calculates  for all  channles in the same sample 
- in instance normalization ==> calculate for one channel in one sample 


## Debugging ML Models 

- Understand bias-variance diagnoses

    - getting more data ==> fixes high variance 
    - smaller set of features ==> fixes high variance 
    

#### Refrence
   - [Prof. Andrew NG Vid](https://www.youtube.com/watch?v=ORrStCArmP4)

## SVM and Kernels

* The main idea of kernels, is that if you can formlate the optimization problem as some of inner products of feater vectors, that can have infinite dimentions, and to come up with a way to calc these inner products efficiently 

we have $ X(i) \in R^{100}$, suppose  W can be expressed as a linear combintaion of X

$ W = \sum_{i = 1}^{M} \alpha_{i} y^i x^i$   (This can be proved with the representer theorem)
- vector W is perpendicular to the decsion boundry specified by algorithm, so W kinds of sets the orientation of the decision boundry and the bias moves it alont right and left.

optimization problem is :
$\min  {w,b}  {1/2} *||W||^2 $  
s.t  $y^i*((W^T * x^i) + b) >= 1$

* For SVM you can make a trade off between the margin and how much you can tolerate wrong calssified examples using a constant 


## Distributed Training  in Pytorch

### Pytorch DDP Internal 

DDP relies on c10d ProcessGroup for communications. Hence, applications must create ProcessGroup instances before constructing DDP.
The DDP constructor takes a reference to the local module, and broadcasts state_dict() from the process with rank 0 to all other processes in the group to make sure that all model replicas start from the exact same state.
DDP registers autograd hooks during construction, one hook per parameter. These hooks will be triggered during the backward pass when the gradient becomes ready.

Backward pass: Because `backward()` function is called on the loss directly, which out of DDP's control. So, it waits till one of the autograd hooks are invoked, to trigger the gradients synchronization.

DDP waits for all gradients in one bucket are ready, Reducer kicks off an asynchronous allreduce on that bucket to calculate mean of gradients across all processes.

Optimizer Step: From the optimizer’s perspective, it is optimizing a local model. Model replicas on all DDP processes can keep in sync because they all start from the same state and they have the same averaged gradients in every iteration.


### DataParallel VS DistributedDataParallel

- `DataParallel` is single-process, multi-thread, and only works on a single machine, while `DistributedDataParallel` is multi-process and works for both single- and multi- machine training. DataParallel is usually slower than DistributedDataParallel even on a single machine due to GIL contention across threads, per-iteration replicated model, and additional overhead introduced by scattering inputs and gathering outputs.
- `DataParallel` doesn't support model parallel 
    






#### Resources 
    - https://pytorch.org/docs/master/notes/ddp.html
    - https://pytorch.org/tutorials/intermediate/ddp_tutorial.html?highlight=distributed%20training